In [1]:
import cv2
import numpy as np
import argparse
import win32api,win32gui,win32ui,win32con
import torch
import time
import pyautogui
import pydirectinput

In [ ]:
# Model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s',channels = 3)
model.to('cuda')
model.conf = 0.4

Using cache found in C:\Users\JPY/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2022-1-6 torch 1.10.1 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)

Fusing layers... 
Model Summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


> 请记得关闭游戏的原始数据输入

# Norm

In [3]:
# 初始化截图区域
window_width = 600
window_height = 600
half_window_width = int(window_width/2)
half_window_height = int(window_height/2)

In [4]:
# 初始化win32截图函数
hwnd = 0  # 窗口的编号，0号表示当前活跃窗口
# 根据窗口句柄获取窗口的设备上下文DC（Divice Context）
hwndDC = win32gui.GetWindowDC(hwnd)
# 根据窗口的DC获取mfcDC
mfcDC = win32ui.CreateDCFromHandle(hwndDC)
# mfcDC创建可兼容的DC
saveDC = mfcDC.CreateCompatibleDC()
# 创建bigmap准备保存图片
saveBitMap = win32ui.CreateBitmap()
# 获取监控器信息
MoniterDev = win32api.EnumDisplayMonitors(None, None)
# w = MoniterDev[0][2][2]
# h = MoniterDev[0][2][3]
# 为bitmap开辟空间
# saveBitMap.CreateCompatibleBitmap(mfcDC, w, h)
saveBitMap.CreateCompatibleBitmap(mfcDC, window_width, window_height)
# 高度saveDC，将截图保存到saveBitmap中
saveDC.SelectObject(saveBitMap)

object 'PyCBitmap' - assoc is 0000023591129AF8, vi=<None>

In [5]:
# 1280,800是我的屏幕的中心坐标
MouseX, MouseY = 1280, 800
# keyboard code
keyboard_dict = {'W':87,'A':65,'S':83,'D':68,'C':67,'H':72}

## V1 敌人进入视野自动攻击

In [6]:
while True:
    # 截取从左上角（1280-half_window_width, 800-half_window_height）长宽为（w，h）的图片
    # saveDC.BitBlt((0, 0), (w, h), mfcDC, (0, 0), win32con.SRCCOPY)
    saveDC.BitBlt((0,0), (window_width, window_height), mfcDC, (1280-half_window_width, 800-half_window_height), win32con.SRCCOPY)
    # saveBitMap.SaveBitmapFile(saveDC, 'filename')
    signedIntsArray = saveBitMap.GetBitmapBits(True)
    im_opencv = np.frombuffer(signedIntsArray, dtype = 'uint8')
    im_opencv.shape = (window_width, window_height, 4)
    imgs = cv2.cvtColor(im_opencv, cv2.COLOR_BGRA2RGB)
    # 运行模型
    results = model(imgs, size=600)
    # 筛选classID == 0
    pd = results.pandas().xyxy[0]  # img1 predictions (pandas)
    pd = pd[pd['class'] == 0]
    if pd.shape[0] == 0:
        continue
    pd['aim_x'] = 0.5*pd['xmin'] + 0.5*pd['xmax']
    pd['aim_y'] = 0.8*pd['ymin'] + 0.2*pd['ymax']
    pd['dis'] = abs(MouseX-pd['aim_x']) + abs(MouseY - pd['aim_y'])
    pd_min = pd.sort_values(by='dis', ascending=True).iloc[0]
    aim_x = pd_min['aim_x'] - half_window_width
    aim_y = pd_min['aim_y'] - half_window_height
    # win32 move mouse
    win32api.SetCursorPos((int(aim_x/1.5)+1280, int(aim_y/1.4)+800))
    # win32api.keybd_event(0x11, 0, 0, 0)
    # win32api.mouse_event(win32con.MOUSEEVENTF_MOVE,int(aim_x/1.5),int(aim_y/1.5))
    # win32 click
    win32api.mouse_event(win32con.MOUSEEVENTF_LEFTDOWN, 0, 0, 0, 0)
    # time.sleep(0.1)
    win32api.mouse_event(win32con.MOUSEEVENTF_LEFTUP, 0, 0, 0, 0)
    # 最小枪口连续子弹时间
    time.sleep(0.15)


## V2 点击后调整瞄准准心
1. 点击一次鼠标后进入模型
2. 长按c鼠标吸附目标
3. 点击H开启/关闭模型

In [ ]:
import os
Run = False
while True:
    # 按键H，开启或关闭脚本
    if win32api.GetAsyncKeyState(72) < 0:
        if Run:
            os.system('close.mp3')
        else:
            os.system('start.mp3')
        Run = not Run
        time.sleep(1)
    if Run == True:
        # 截取从左上角（1280-half_window_width, 800-half_window_height）长宽为（w，h）的图片
        # saveDC.BitBlt((0, 0), (w, h), mfcDC, (0, 0), win32con.SRCCOPY)
        saveDC.BitBlt((0,0), (window_width, window_height), mfcDC, (1280-half_window_width, 800-half_window_height), win32con.SRCCOPY)
        # saveBitMap.SaveBitmapFile(saveDC, 'filename')
        signedIntsArray = saveBitMap.GetBitmapBits(True)
        im_opencv = np.frombuffer(signedIntsArray, dtype = 'uint8')
        im_opencv.shape = (window_width, window_height, 4)
        imgs = cv2.cvtColor(im_opencv, cv2.COLOR_BGRA2RGB)
        # 点击鼠标左键后瞄头
        if win32api.GetAsyncKeyState(win32con.MK_LBUTTON) < 0:
            # 运行模型
            results = model(imgs, size=600)
            # 筛选classID == 0
            pd = results.pandas().xyxy[0]  # img1 predictions (pandas)
            pd = pd[pd['class'] == 0]
            if pd.shape[0] == 0:
                continue
            pd['aim_x'] = 0.5*pd['xmin'] + 0.5*pd['xmax']
            pd['aim_y'] = 0.8*pd['ymin'] + 0.2*pd['ymax']
            pd['dis'] = abs(MouseX-pd['aim_x']) + abs(MouseY - pd['aim_y'])
            pd_min = pd.sort_values(by='dis', ascending=True).iloc[0]
            aim_x = pd_min['aim_x'] - half_window_width
            aim_y = pd_min['aim_y'] - half_window_height
            # win32 move mouse
            win32api.SetCursorPos((int(aim_x/1.5)+1280, int(aim_y/1.4)+800))
            # win32api.keybd_event(0x11, 0, 0, 0)
            # win32api.mouse_event(win32con.MOUSEEVENTF_MOVE,int(aim_x/1.5),int(aim_y/1.5))
            # win32 click
            time.sleep(0.07)
            win32api.mouse_event(win32con.MOUSEEVENTF_LEFTDOWN, 0, 0, 0, 0)
            # time.sleep(0.1)
            win32api.mouse_event(win32con.MOUSEEVENTF_LEFTUP, 0, 0, 0, 0)
            # 最小枪口连续子弹时间
            # time.sleep(0.05)
        # 按住‘c'键，自动吸附
        elif win32api.GetAsyncKeyState(67) < 0:
            # 运行模型
            results = model(imgs, size=600)
            # 筛选classID == 0
            pd = results.pandas().xyxy[0]  # img1 predictions (pandas)
            pd = pd[pd['class'] == 0]
            if pd.shape[0] == 0:
                continue
            pd['aim_x'] = 0.5*pd['xmin'] + 0.5*pd['xmax']
            pd['aim_y'] = 0.8*pd['ymin'] + 0.2*pd['ymax']
            pd['dis'] = abs(MouseX-pd['aim_x']) + abs(MouseY - pd['aim_y'])
            pd_min = pd.sort_values(by='dis', ascending=True).iloc[0]
            aim_x = pd_min['aim_x'] - half_window_width
            aim_y = pd_min['aim_y'] - half_window_height
            # win32 move mouse
            win32api.SetCursorPos((int(aim_x/1.6)+1280, int(aim_y/1.6)+800))
            time.sleep(0.05)
        else:
            continue

## V3 norm+按钮开关脚本
1. 键入H键开启/关闭模型
2. 敌人出现在视野中将自动攻击

In [ ]:
Run = False
while True:
    if win32api.GetAsyncKeyState(72) < 0:
        if Run:
            os.system('close.mp3')
        else:
            os.system('start.mp3')
        Run = not Run
        time.sleep(1)
    if Run == True:
        # 截取从左上角（1280-half_window_width, 800-half_window_height）长宽为（w，h）的图片
        # saveDC.BitBlt((0, 0), (w, h), mfcDC, (0, 0), win32con.SRCCOPY)
        saveDC.BitBlt((0,0), (window_width, window_height), mfcDC, (1280-half_window_width, 800-half_window_height), win32con.SRCCOPY)
        # saveBitMap.SaveBitmapFile(saveDC, 'filename')
        signedIntsArray = saveBitMap.GetBitmapBits(True)
        im_opencv = np.frombuffer(signedIntsArray, dtype = 'uint8')
        im_opencv.shape = (window_width, window_height, 4)
        imgs = cv2.cvtColor(im_opencv, cv2.COLOR_BGRA2RGB)
        # 运行模型
        results = model(imgs, size=600)
        # 筛选classID == 0
        pd = results.pandas().xyxy[0]  # img1 predictions (pandas)
        pd = pd[pd['class'] == 0]
        if pd.shape[0] == 0:
            continue
        pd['aim_x'] = 0.5*pd['xmin'] + 0.5*pd['xmax']
        pd['aim_y'] = 0.8*pd['ymin'] + 0.2*pd['ymax']
        pd['dis'] = abs(MouseX-pd['aim_x']) + abs(MouseY - pd['aim_y'])
        pd_min = pd.sort_values(by='dis', ascending=True).iloc[0]
        aim_x = pd_min['aim_x'] - half_window_width
        aim_y = pd_min['aim_y'] - half_window_height
        # win32 move mouse
        win32api.SetCursorPos((int(aim_x/1.5)+1280, int(aim_y/1.4)+800))
        # win32api.keybd_event(0x11, 0, 0, 0)
        # win32api.mouse_event(win32con.MOUSEEVENTF_MOVE,int(aim_x/1.5),int(aim_y/1.5))
        # win32 click
        win32api.mouse_event(win32con.MOUSEEVENTF_LEFTDOWN, 0, 0, 0, 0)
        # time.sleep(0.1)
        win32api.mouse_event(win32con.MOUSEEVENTF_LEFTUP, 0, 0, 0, 0)
        # 最小枪口连续子弹时间
        time.sleep(0.1)
    else:
        continue

## V4 V2+急停实现
[x] 点击一次鼠标后进入模型  
[x] 长按c鼠标吸附目标  
[x] 点击H开启/关闭模型  
[x] 急停实现 基本实现，但仍需玩家松开适时松开方向键   
[x] 非一帧拉枪  实现效果不佳

In [6]:
# 实现非一帧拉枪
def move_aim(times, xoff, yoff):
    x_temp = xoff / times
    y_temp = yoff / times
    x = [x_temp]*(times-1)
    x.append(xoff - x_temp*(times-1))
    y = [y_temp]*(times-1)
    y.append(yoff - y_temp*(times-1))
    for i in range(times):
        win32api.SetCursorPos((int(x[i]/(1.5*times))+1280, int(y[i]/(1.4*times))+800))
        time.sleep(0.02)

In [7]:
# 实现急停
def peek():
    # 分情况讨论，WA,WD,AS,SD,W,A,S,D
    # 检测按键WA
    if win32api.GetAsyncKeyState(keyboard_dict['W']) < 0 and win32api.GetAsyncKeyState(keyboard_dict['A']) < 0 :
        win32api.keybd_event(keyboard_dict['W'], 0, win32con.KEYEVENTF_KEYUP, 0)
        win32api.keybd_event(keyboard_dict['A'], 0, win32con.KEYEVENTF_KEYUP, 0)
        # 按下S与D
        win32api.keybd_event(keyboard_dict['S'],0,0,0)
        win32api.keybd_event(keyboard_dict['D'],0,0,0)
        win32api.keybd_event(keyboard_dict['S'], 0, win32con.KEYEVENTF_KEYUP, 0)
        win32api.keybd_event(keyboard_dict['D'], 0, win32con.KEYEVENTF_KEYUP, 0)
    # WD
    elif win32api.GetAsyncKeyState(keyboard_dict['W']) < 0 and win32api.GetAsyncKeyState(keyboard_dict['D']) < 0 :
        win32api.keybd_event(keyboard_dict['W'], 0, win32con.KEYEVENTF_KEYUP, 0)
        win32api.keybd_event(keyboard_dict['D'], 0, win32con.KEYEVENTF_KEYUP, 0)
        # 按下S与D
        win32api.keybd_event(keyboard_dict['A'],0,0,0)
        win32api.keybd_event(keyboard_dict['S'],0,0,0)
        win32api.keybd_event(keyboard_dict['A'], 0, win32con.KEYEVENTF_KEYUP, 0)
        win32api.keybd_event(keyboard_dict['S'], 0, win32con.KEYEVENTF_KEYUP, 0)
    # AS
    elif win32api.GetAsyncKeyState(keyboard_dict['A']) < 0 and win32api.GetAsyncKeyState(keyboard_dict['S']) < 0 :
        win32api.keybd_event(keyboard_dict['A'], 0, win32con.KEYEVENTF_KEYUP, 0)
        win32api.keybd_event(keyboard_dict['S'], 0, win32con.KEYEVENTF_KEYUP, 0)
        # 按下S与D
        win32api.keybd_event(keyboard_dict['W'],0,0,0)
        win32api.keybd_event(keyboard_dict['D'],0,0,0)
        win32api.keybd_event(keyboard_dict['W'], 0, win32con.KEYEVENTF_KEYUP, 0)
        win32api.keybd_event(keyboard_dict['D'], 0, win32con.KEYEVENTF_KEYUP, 0)
    # SD
    elif win32api.GetAsyncKeyState(keyboard_dict['S']) < 0 and win32api.GetAsyncKeyState(keyboard_dict['D']) < 0 :
        win32api.keybd_event(keyboard_dict['S'], 0, win32con.KEYEVENTF_KEYUP, 0)
        win32api.keybd_event(keyboard_dict['D'], 0, win32con.KEYEVENTF_KEYUP, 0)
        # 按下S与D
        win32api.keybd_event(keyboard_dict['W'],0,0,0)
        win32api.keybd_event(keyboard_dict['A'],0,0,0)
        win32api.keybd_event(keyboard_dict['W'], 0, win32con.KEYEVENTF_KEYUP, 0)
        win32api.keybd_event(keyboard_dict['A'], 0, win32con.KEYEVENTF_KEYUP, 0)
    # W
    elif win32api.GetAsyncKeyState(keyboard_dict['W']) < 0:
        win32api.keybd_event(keyboard_dict['W'], 0, win32con.KEYEVENTF_KEYUP, 0)
        # 按下S与D
        win32api.keybd_event(keyboard_dict['S'],0,0,0)
        win32api.keybd_event(keyboard_dict['S'], 0, win32con.KEYEVENTF_KEYUP, 0)
    # A
    elif win32api.GetAsyncKeyState(keyboard_dict['A']) < 0:
        win32api.keybd_event(keyboard_dict['A'], 0, win32con.KEYEVENTF_KEYUP, 0)
        # 按下S与D
        win32api.keybd_event(keyboard_dict['D'],0,0,0)
        win32api.keybd_event(keyboard_dict['D'], 0, win32con.KEYEVENTF_KEYUP, 0)
    # S
    elif win32api.GetAsyncKeyState(keyboard_dict['S']) < 0:
        win32api.keybd_event(keyboard_dict['S'], 0, win32con.KEYEVENTF_KEYUP, 0)
        # 按下S与D
        win32api.keybd_event(keyboard_dict['W'],0,0,0)
        win32api.keybd_event(keyboard_dict['W'], 0, win32con.KEYEVENTF_KEYUP, 0)
    # D
    elif win32api.GetAsyncKeyState(keyboard_dict['D']) < 0:
        win32api.keybd_event(keyboard_dict['D'], 0, win32con.KEYEVENTF_KEYUP, 0)
        # 按下S与D
        win32api.keybd_event(keyboard_dict['A'],0,0,0)
        win32api.keybd_event(keyboard_dict['A'], 0, win32con.KEYEVENTF_KEYUP, 0)


In [10]:
import os
Run = False
while True:
    # 按键H，开启或关闭脚本
    if win32api.GetAsyncKeyState(72) < 0:
        if Run:
            os.system('close.mp3')
        else:
            os.system('start.mp3')
        Run = not Run
        time.sleep(1)
    if Run == True:
        # 截取从左上角（1280-half_window_width, 800-half_window_height）长宽为（w，h）的图片
        # saveDC.BitBlt((0, 0), (w, h), mfcDC, (0, 0), win32con.SRCCOPY)
        saveDC.BitBlt((0,0), (window_width, window_height), mfcDC, (1280-half_window_width, 800-half_window_height), win32con.SRCCOPY)
        # saveBitMap.SaveBitmapFile(saveDC, 'filename')
        signedIntsArray = saveBitMap.GetBitmapBits(True)
        im_opencv = np.frombuffer(signedIntsArray, dtype = 'uint8')
        im_opencv.shape = (window_width, window_height, 4)
        imgs = cv2.cvtColor(im_opencv, cv2.COLOR_BGRA2RGB)
        # 点击鼠标左键后瞄头
        if win32api.GetAsyncKeyState(win32con.MK_LBUTTON) < 0:
            # 运行模型
            results = model(imgs, size=600)
            # 筛选classID == 0
            pd = results.pandas().xyxy[0]  # img1 predictions (pandas)
            pd = pd[pd['class'] == 0]
            if pd.shape[0] == 0:
                continue
            pd['aim_x'] = 0.5*pd['xmin'] + 0.5*pd['xmax']
            pd['aim_y'] = 0.8*pd['ymin'] + 0.2*pd['ymax']
            pd['dis'] = abs(MouseX-pd['aim_x']) + abs(MouseY - pd['aim_y'])
            pd_min = pd.sort_values(by='dis', ascending=True).iloc[0]
            aim_x = pd_min['aim_x'] - half_window_width
            aim_y = pd_min['aim_y'] - half_window_height
            # 急停
            peek()
            # win32 move mouse
            # AK para 1.3 & 1.4
            win32api.SetCursorPos((int(aim_x/1.3)+1280, int(aim_y/1.4)+800))
            # 分n段到达瞄准位置
            # move_aim(3,aim_x,aim_y)
            # win32api.keybd_event(0x11, 0, 0, 0)
            # win32api.mouse_event(win32con.MOUSEEVENTF_MOVE,int(aim_x/1.5),int(aim_y/1.5))
            # win32 click
            time.sleep(0.07)
            # win32api.mouse_event(win32con.MOUSEEVENTF_LEFTDOWN, 0, 0, 0, 0)
            # time.sleep(0.1)
            # win32api.mouse_event(win32con.MOUSEEVENTF_LEFTUP, 0, 0, 0, 0)
            # 最小枪口连续子弹时间
            # time.sleep(0.05)
        # 按住‘c'键，自动吸附
        elif win32api.GetAsyncKeyState(67) < 0:
            # 运行模型
            results = model(imgs, size=600)
            # 筛选classID == 0
            pd = results.pandas().xyxy[0]  # img1 predictions (pandas)
            pd = pd[pd['class'] == 0]
            if pd.shape[0] == 0:
                continue
            pd['aim_x'] = 0.5*pd['xmin'] + 0.5*pd['xmax']
            pd['aim_y'] = 0.8*pd['ymin'] + 0.2*pd['ymax']
            pd['dis'] = abs(MouseX-pd['aim_x']) + abs(MouseY - pd['aim_y'])
            pd_min = pd.sort_values(by='dis', ascending=True).iloc[0]
            aim_x = pd_min['aim_x'] - half_window_width
            aim_y = pd_min['aim_y'] - half_window_height
            # win32 move mouse
            win32api.SetCursorPos((int(aim_x/1.6)+1280, int(aim_y/1.6)+800))
            time.sleep(0.05)
        else:
            continue

# 动量预测

同时处理两张图片，通过两张图片的aim_position预测出接下去一个时间步的aim_position,用于瞄准在移动中的敌人

In [ ]:
intel_window_size = 480
half_intel_window_size = int(intel_window_size/2)
# 截图
hwnd = 0  # 窗口的编号，0号表示当前活跃窗口
# 根据窗口句柄获取窗口的设备上下文DC（Divice Context）
hwndDC = win32gui.GetWindowDC(hwnd)
# 根据窗口的DC获取mfcDC
mfcDC = win32ui.CreateDCFromHandle(hwndDC)
# mfcDC创建可兼容的DC
saveDC = mfcDC.CreateCompatibleDC()
# 创建bigmap准备保存图片
saveBitMap = win32ui.CreateBitmap()
# 获取监控器信息
MoniterDev = win32api.EnumDisplayMonitors(None, None)
# w = MoniterDev[0][2][2]
# h = MoniterDev[0][2][3]
two_aims = [[None,None],[None,None]]
while True:
    t1 = time.time()
    MouseX, MouseY = pyautogui.position()
    # 为bitmap开辟空间
    # saveBitMap.CreateCompatibleBitmap(mfcDC, w, h)
    saveBitMap.CreateCompatibleBitmap(mfcDC, intel_window_size, intel_window_size)
    # 高度saveDC，将截图保存到saveBitmap中
    saveDC.SelectObject(saveBitMap)
    # 截取从左上角（0，0）长宽为（w，h）的图片
    # saveDC.BitBlt((0, 0), (w, h), mfcDC, (0, 0), win32con.SRCCOPY)
    saveDC.BitBlt((0,0), (intel_window_size,intel_window_size), mfcDC, (1280-half_intel_window_size, 800-half_intel_window_size), win32con.SRCCOPY)
    # saveBitMap.SaveBitmapFile(saveDC, 'filename')
    signedIntsArray = saveBitMap.GetBitmapBits(True)
    im_opencv = np.frombuffer(signedIntsArray, dtype = 'uint8')
    im_opencv.shape = (intel_window_size, intel_window_size, 4)
    imgs = cv2.cvtColor(im_opencv, cv2.COLOR_BGRA2RGB)
    
    t2 = time.time()
    print('图像准备时间：',t2-t1)

    results = model(imgs, size=intel_window_size)

    t3 = time.time()
    print('模型预测时间：',t3-t2)

    pd = results.pandas().xyxy[0]  # img1 predictions (pandas)
    pd = pd[pd['class'] == 0]
    if pd.shape[0] == 0:
        # 重新初始化
        two_aims = [[None,None],[None,None]]
        continue
    pd['aim_x'] = 0.48*pd['xmin'] + 0.52*pd['xmax']
    pd['aim_y'] = 0.9*pd['ymin'] + 0.1*pd['ymax']
    pd['dis'] = abs(MouseX-pd['aim_x']) + abs(MouseY - pd['aim_y'])
    pd_min = pd.sort_values(by='dis', ascending=True).iloc[0]
    aim_x = pd_min['aim_x'] - half_intel_window_size
    aim_y = pd_min['aim_y'] - half_intel_window_size
    two_aims[0],two_aims[1] = two_aims[1],[aim_x,aim_y]

    t4 = time.time()
    print('计算aim时间:',t4-t3)

    if two_aims[0] == [None,None]:
        pydirectinput.moveTo(int(aim_x/1.5)+1280, int(aim_y/1.5)+800,duration=0)
        pyautogui.click(clicks = 1,interval=0.02)
    else:
        aimx_1,aimy_1 = two_aims[0]
        aimx_2,aimy_2 = two_aims[1]
        distance = abs(aimx_1-aimx_2) + abs(aimy_1-aimy_2)
        if distance > 25:
            continue
            pydirectinput.moveTo(int(aim_x/1.5)+1280, int(aim_y/1.5)+800,duration=0)
            pyautogui.click(clicks = 1,interval=0.02)
        else:
            delta_x = aimx_2 - aimx_1
            delta_y = aimy_2 - aimy_1
            aim_x = aimx_2 + 6*delta_x
            aim_y = aimy_2 + 2*delta_y
            pydirectinput.moveTo(int(aim_x/1.5)+1280, int(aim_y/1.5)+800,duration=0)
            pyautogui.click(clicks = 1,interval=0.02)
    t5 = time.time()
    print('click时间',t5-t4)
    break
    # pydirectinput.moveTo(int(aim_x)+1280, int(aim_y)+800,duration=0)
    # pydirectinput.click()
    # pyautogui.click(clicks = 1,duration=0.03)
    # pydirectinput.click()

# win32api test

In [ ]:
while True:
    t1=time.time()
    win32api.mouse_event(win32con.MOUSEEVENTF_MOVE,-10,-10)
    win32api.mouse_event(win32con.MOUSEEVENTF_LEFTDOWN, 0, 0, 0, 0)
    win32api.mouse_event(win32con.MOUSEEVENTF_LEFTUP, 0, 0, 0, 0)
    t2 = time.time()
    print(t2-t1)
    time.sleep(2)

In [ ]:
keyboard_dict = {'w':87,'A':65,'S':83,'D':68}

In [ ]:
while True:
    if win32api.GetAsyncKeyState(win32con.MK_LBUTTON) == 1:
        print('tracking mouse down')
    if win32api.GetKeyState(win32con.MK_LBUTTON) < 0:
        print('tracking mouse down - ')
    time.sleep(0.5) 

In [ ]:
for i in range(20):
    time.sleep(1)
    # print(win32api.GetKeyState(87))
    # print(win32api.GetAsyncKeyState(win32con.MK_LBUTTON))
    print(win32api.GetAsyncKeyState(87))
    if win32api.GetAsyncKeyState(87) < 0:
        win32api.keybd_event(87, 0, win32con.KEYEVENTF_KEYUP, 0)
    print(win32api.GetAsyncKeyState(87))
    